Data Glacier 

In [24]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from matplotlib import pyplot as plt
import seaborn as sns
%matplotlib inline
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
data = pd.read_csv('Cab_Data.csv')
cities = pd.read_csv('City_Data.csv')
customers = pd.read_csv('Customer_ID.csv')
trx = pd.read_csv('Transaction_ID.csv')
data.head()
cities.head()
customers.head()
trx.head()
print(data.shape)
print(cities.shape)
print(customers.shape)
print(trx.shape)
newdata = data.set_index('Transaction ID').join(trx.set_index('Transaction ID'))
newdata.head()
new = pd.concat(customers.set_index('Customer ID'), on ='Customer ID')
new.head()
# Adding The Gross Column to the dataframe
new['Gross'] = new['Price Charged'] - new['Cost of Trip']
new.reset_index(inplace=True)
new.head()
#pip install xlrd
# xlrd is a library for reading data and formatting information from Excel files in the historical .xls format.
# I used it here to convert the 'Date of Travel' column to datetime format
import xlrd
  
new['Date of Travel'] = new['Date of Travel'].apply(
    lambda s: xlrd.xldate.xldate_as_datetime(s, 0)
                                                    )
new.head()
new.shape

# Checking whether there's is Null Values
new.isnull().sum()
# Creating two minor tables for each company
yelw_cities = new.loc[new['Company']=='Yellow Cab']
pnk_cities = new.loc[new['Company']=='Pink Cab']
yelw_cities.reset_index(inplace=True)
pnk_cities.reset_index(inplace=True)
print(yelw_cities.shape[0])
print(pnk_cities.shape[0])
# Transactions' total Num for each Company

ax = sns.catplot(y="Company", kind="count",palette=['pink','yellow'], data=new)
ax.set_axis_labels('Total Transactions No.', '')
ax.fig.suptitle('Transaction No.')
# Cost of daily Trips comparison between the Two Companies

# Grouping Cost and Dates for Ploting and analysis
yelw_cost =yelw_cities.groupby(['Date of Travel'])['Cost of Trip'].sum()
pnk_cost =pnk_cities.groupby(['Date of Travel'])['Cost of Trip'].sum()
list_of_dates=new.groupby(['Date of Travel']).sum().reset_index()['Date of Travel']

# Plotting:


plt.figure(figsize=(25,12))
plt.plot(list_of_dates,yelw_cost, color='yellow')
plt.plot(list_of_dates,pnk_cost,color='pink')
plt.xlabel('Dates',fontsize=20)
plt.ylabel('Cost of the trips',fontsize=20)
plt.title(' Cost of Trips Comparison on Daily basis',fontsize=25)
plt.xticks(rotation=45)
plt.show()

# Gross of daily Trips comparison between the Two Companies

# Grouping Gross and Dates for Ploting and analysis
yelw_gross =yelw_cities.groupby(['Date of Travel'])['Gross'].sum()
pnk_gross =pnk_cities.groupby(['Date of Travel'])['Gross'].sum()

# Plotting:
plt.figure(figsize=(25,12))
plt.plot(list_of_dates,yelw_gross, color='yellow')
plt.plot(list_of_dates,pnk_gross,color='pink')
plt.xlabel('Dates',fontsize=20)
plt.ylabel('Gross',fontsize=20)
plt.title(' Daily Gross Comparison',fontsize=25)
plt.xticks(rotation=45)
plt.show()

# Daily  comparison between the Two Companies in Number of Transaction

# Grouping Gross and Dates for Ploting and analysis
yelw_trans =yelw_cities.groupby(['Date of Travel'])['Transaction ID'].count()
pnk_trans =pnk_cities.groupby(['Date of Travel'])['Transaction ID'].count()

# Plotting:


plt.figure(figsize=(25,12))
plt.plot(list_of_dates,yelw_trans, color='yellow')
plt.plot(list_of_dates,pnk_trans,color='pink')
plt.xlabel('Dates',fontsize=20)
plt.ylabel('Transaction/s',fontsize=20)
plt.title(' Daily Transactions Num.',fontsize=25)
plt.xticks(rotation=45)
plt.show()

# Yellow Cab presence in the cities according to its transactions number

plt.figure(figsize=(25,12))
plt.xticks(rotation=45)
yel_cities= sns.countplot(x='City', data = yelw_cities,
                          order= yelw_cities.groupby('City')['City'].count().sort_values(ascending=False).index.values,
                          palette=['yellow'])
plt.ylabel('Transaction/s',fontsize=20)
plt.title('Transactions Num.',fontsize=25)
plt.show()

# Pink Cab presence in the cities according to its transactions number

plt.figure(figsize=(25,12))
plt.xticks(rotation=45)
pk_cities= sns.countplot(x='City', data = pnk_cities,
                          order= pnk_cities.groupby('City')['City'].count().sort_values(ascending=False).index.values,
                          palette=['pink'])
plt.ylabel('Transaction/s',fontsize=20)
plt.title('Transactions Num.',fontsize=25)
plt.show()

# The correlation between No. of Customers & Gross in Yellow Cab

# Grouping Customer Num. According to the dates
yelw_cus = yelw_cities.groupby(['Date of Travel'])['Customer ID'].count()


# Scatter Plot
plt.figure(figsize=(25,12))
plt.scatter(yelw_cus,yelw_gross)
plt.ylabel('Gross',fontsize=20)
plt.xlabel('Num. of Customers',fontsize=20)
plt.title('The relation Between Gross and Customers number in Yellow Cab',fontsize=25)
plt.xticks(rotation=45)
plt.show()

# The correlation between No. of Customers & Gross in Pink Cab

# Grouping Customer Num. According to the dates
pink_cus = pnk_cities.groupby(['Date of Travel'])['Customer ID'].count()


# Scatter Plot
plt.figure(figsize=(25,12))
plt.scatter(pink_cus,pnk_gross,color='r')
plt.ylabel('Gross',fontsize=20)
plt.xlabel('Num. of Customers',fontsize=20)
plt.title('The relation Between Gross and Customers number in Pink Cab',fontsize=25)
plt.xticks(rotation=45)
plt.show()

# Conclusion for the relation between Cutomers and Gross
print('The correlation in Yellow cab between Customers Num. and Gross is:', np.corrcoef(yelw_cus,yelw_gross)[1,0])
print('The correlation in Pink cab between Customers Num. and Gross is:', np.corrcoef(pink_cus,pnk_gross)[1,0])

# Cusromers attributes Exploratory

#Age distribution between males and females in both companies

# Droping duplicates of Customer ID's for average age calcuation
no_cust_dup =new.drop_duplicates(subset=['Customer ID'])
print(no_cust_dup.shape[0])

# Plotting Age
plt.figure(figsize=(9,8))
sns.boxplot(data=no_cust_dup, x='Company', y='Age',hue='Gender',palette=['blue','purple'])
plt.title('Age Distribution according to the Gender in both of the Companies',fontsize=16)
plt.xlabel('Gender',fontsize=15)
plt.ylabel('Age',fontsize=15)
plt.show()

# Income Exploratory

# Plotting Income
plt.figure(figsize=(9,8))
sns.boxplot(data=no_cust_dup, x='Company', 
            y='Income (USD/Month)',
            hue='Payment_Mode',
            palette=['grey','green'])
plt.title('Income Exploratory ',fontsize=18)
plt.show()

# The average age of the Customers
no_cust_dup.groupby(['Company','Gender'])['Age'].agg('mean').round(0)

# Common Customers Exploratory and unique Customers
yel_cust_uniq = np.asarray(np.unique(yelw_cities['Customer ID']))
pnk_cust_uniq = np.asarray(np.unique(pnk_cities['Customer ID']))
customers_intersection = len(np.intersect1d(yel_cust_uniq, pnk_cust_uniq))
print('Unqiue total Customers Num. in Yellow Cab is:',len(yel_cust_uniq))
print('Unqiue total Customers Num. in Pink Cab is:',len(pnk_cust_uniq))
print('The common Num. of Customers between the two Companies is: {} Customer'.format(customers_intersection))

# The average income of the Customers
no_cust_dup.groupby('Company')['Income (USD/Month)'].agg('mean').round(0)

# Payment Mode Counts in each Company
no_cust_dup.groupby(['Company','Payment_Mode'])['Payment_Mode'].agg('count')

# What do the genders prefer to ride with:
no_cust_dup.groupby(['Gender','Company'])['Company'].agg('count')

(359392, 7)
(20, 3)
(49171, 4)
(205013, 3)


TypeError: ignored